### Data Exploration
The recipes dataset contains 522,517 recipes from 312 different categories. This dataset provides information about each recipe like cooking times, servings, ingredients, nutrition, instructions, and more.

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("../dataset/recipes.csv")

In [3]:
dataset.head(2)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."


In [4]:
dataset.shape

(522517, 28)

In [5]:
dataset.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

In [6]:
dataset.isnull().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
DatePublished                      0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                       17237
RecipeIngredientQuantities         3
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeYield                   348071
R

### Data Preprocessing

In [7]:
features = dataset[['Name', 'RecipeIngredientQuantities', 'RecipeIngredientParts','RecipeInstructions']]

In [8]:
features.head(3)

,Name,RecipeIngredientQuantities,RecipeIngredientParts,RecipeInstructions
0,Low-Fat Berry Blue Frozen Dessert,"c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...","c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,Biryani,"c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...","c(""Soak saffron in warm milk for 5 minutes and..."
2,Best Lemonade,"c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...","c(""Into a 1 quart Jar with tight fitting lid, ..."


In [9]:
full_text = features.loc[0, 'RecipeIngredientParts']
print(full_text)

c("blueberries", "granulated sugar", "vanilla yogurt", "lemon juice")


In [10]:
# Converting from R-style list to Python List
def to_python_list(r_list_string):
    if(isinstance(r_list_string, str)):
        return r_list_string.strip('c()').replace('"', '').split(', ')
    return []

In [11]:
test = to_python_list(full_text)
print(test)

['blueberries', 'granulated sugar', 'vanilla yogurt', 'lemon juice']


In [12]:
features.loc[:, 'RecipeIngredientParts'] = features['RecipeIngredientParts'].apply(to_python_list)

In [13]:
features.loc[:, 'RecipeInstructions'] = features['RecipeInstructions'].apply(to_python_list)
features.loc[:, 'RecipeIngredientQuantities'] = features['RecipeIngredientQuantities'].apply(to_python_list)

In [14]:
features.head(2)

,Name,RecipeIngredientQuantities,RecipeIngredientParts,RecipeInstructions
0,Low-Fat Berry Blue Frozen Dessert,"[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...","[Toss 2 cups berries with sugar., Let stand fo..."
1,Biryani,"[1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1...","[saffron, milk, hot green chili peppers, onion...",[Soak saffron in warm milk for 5 minutes and p...


#### Score Similarity Each Recipe (v1)
1. Simple, fast
2. Can’t handle fuzzy matches like "garlic cloves" vs "garlic"
3. Doesn’t consider importance of ingredients

In [15]:
def ingredient_match_score_v1(recipe_ingredients, user_ingredients):
    return len(set(recipe_ingredients).intersection(set(user_ingredients)))

In [16]:
user_ingredients = ["onion", "garlic", "tomato"]

In [17]:
features_v1 = features.copy()  # Make an explicit copy
features_v1['match_score'] = features_v1['RecipeIngredientParts'].apply(
    lambda recipe: ingredient_match_score_v1(recipe, user_ingredients)
)

In [18]:
top_recipes = features_v1.sort_values(by='match_score', ascending=False)
best_match = top_recipes.iloc[0]

In [19]:
best_match

Name                                               Blumenthal's Small Frittatas
RecipeIngredientQuantities    [2, 1/2, 1/2, NA, 1/2, 1, 4, 2, 1/4, 1/8, 1/8,...
RecipeIngredientParts         [frozen hash browns, cheddar cheese, sausage, ...
RecipeInstructions            [Preheat over to 325 degrees., Grease (or use ...
match_score                                                                   2
Name: 13848, dtype: object

In [20]:
print("Name: ", best_match['Name'])
print("Ingredients:", best_match['RecipeIngredientParts'])
print("\nInstructions:")
for step in best_match['RecipeInstructions']:
    print("-", step)

Name:  Blumenthal's Small Frittatas
Ingredients: ['frozen hash browns', 'cheddar cheese', 'sausage', 'olive oil', 'onion', 'garlic', 'eggs', 'oregano', 'marjoram', 'sage', 'tarragon', 'plain yogurt', 'biscuit mix']

Instructions:
- Preheat over to 325 degrees.
- Grease (or use paper liners) 12 muffin tins.
- Saute onions and garlic in olive oil.
- Cook sausage.
- Microwave or steam-cook potatoes (do not overcook).
- Toss onions
- garlic
- sausage
- and potatoes with cheese.
- Divide evenly into 12 tins.
- Whip eggs
- egg whites
- and fajita seasoning.
- Mix in other seasonings.
- Pour mixture evenly over potatoe mixture.
- Bake 25 minutes.
- Turn oven off
- but let sit in oven 10 minutes.
- Serve warm.
- Mix in yogurt and biscuit mix.



#### TF-IDF + Cosine Similarity (Score Similarity v2)
1. **Term Frequency** – how often a word appears in a document
2. **Inverse Document Frequency** – how rare that word is across all documents

##### Pros:
1. Matches exact words
2. Handles frequent/common words
3. Can rank results better
4. Learns from the whole dataset
5. Weighs ingredients by importance
6. Finds smarter matches than just counting overlaps
7. Is scalable and easy to expand

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [22]:
features_v2 = features.copy()

In [23]:
# Each recipe’s ingredient list becomes a “document”:
# ["chicken", "salt", "garlic"] → "chicken salt garlic"
corpus = features_v2['RecipeIngredientParts'].apply(lambda parts: ' '.join(parts))

In [24]:
# Vectorize each recipe's ingredients

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

In [25]:
user_query = "onion garlic tomato"

In [26]:
# Vectorize the user's ingredients
user_vec = vectorizer.transform([user_query])

In [27]:
# Use cosine similarity to compare
cos_sim = cosine_similarity(user_vec, tfidf_matrix)

In [28]:
top_idx = np.argmax(cos_sim)
top_recipe = features.iloc[top_idx]

In [29]:
print("Name: ", top_recipe['Name'])
print("Top recipe match:")
print("\nIngredients:", top_recipe['RecipeIngredientParts'])

print("\nInstructions:")
for step in top_recipe['RecipeInstructions']:
    print("-", step)

Name:  Mexican Bean Dip
Top recipe match:

Ingredients: ['tomato sauce', 'onion']

Instructions:
- Place all ingredients in a medium Microwave-safe bowl.
- Mix well.
- Cover with plastic wrap.
- Microwave on High 6 minutes.
- Stir at 3 minutes.
- Serve with corn chips.


#### Fuzzy Matching Similarity (Score Similarity V3)
1. Recognize that "roma tomato" ≈ "tomato"
2. Handle typos or partial matches (like "onions" vs "onion")
3. Improve ingredient matching without using heavy models

In [30]:
from rapidfuzz import fuzz

In [31]:
def fuzzy_match_score(recipe_ingredients, user_ingred):
    total_score = 0

    for user_ing in user_ingred:
        # For each user ingredient, find best fuzzy match in the recipe
        best = max(fuzz.partial_ratio(user_ing, recipe_ing) for recipe_ing in recipe_ingredients)
        total_score += best
    
    # Normalizing the score
    return total_score / len(user_ingred) if user_ingred else 0

In [32]:
user_ingredients = ["roma tomato", "onion", "garlic"]

In [33]:
features_v3 = features.copy()

In [34]:
features_v3['fuzzy_score'] = features_v3['RecipeIngredientParts'].apply(
    lambda recipe: fuzzy_match_score(recipe, user_ingredients)
)

In [39]:
# Top match (highest Fuzzy Score)
top_recipe = features_v3.sort_values(by='fuzzy_score', ascending=False).iloc[0]

In [40]:
print("Top Fuzzy Match Name:", top_recipe['Name'])
print("Ingredients:", top_recipe['RecipeIngredientParts'])
print("\nInstructions:")
for step in top_recipe['RecipeInstructions']:
    print("-", step)

Top Fuzzy Match Name: Fatoosh (Middle Eastern Bread Salad)
Ingredients: ['romaine lettuce', 'green onions', 'cucumber', 'roma tomatoes', 'fresh parsley', 'garlic cloves', 'dried mint', 'olive oil', 'salt']

Instructions:
- Toast the pita breads and with a scissors
- cut into bite sized pieces.
- Set aside.
- Toss lettuce
- green onion
- cucumber
- tomatoes
- parsley
- garlic and mint in a large mixing bowl.
- Add lemon juice
- olive oil and sumac.
- Toss.
- Add bread and then salt to taste.
- Best if allowed to chill for a few hours before serving.
- sumac is a maroonish colored middle eastern spice that can be found almost anywhere.  It adds a nice color and flavor.
- It's not necessary
- but a very nice touch to this recipe.



In [42]:
top_5_recipes = features_v3.sort_values(by='fuzzy_score', ascending=False).head(5)

In [43]:
for idx, row in top_5_recipes.iterrows():
    print("Top Fuzzy Match Name:", row['Name'])
    print("Ingredients:", row['RecipeIngredientParts'])
    print("-" * 40)

Top Fuzzy Match Name: Fatoosh (Middle Eastern Bread Salad)
Ingredients: ['romaine lettuce', 'green onions', 'cucumber', 'roma tomatoes', 'fresh parsley', 'garlic cloves', 'dried mint', 'olive oil', 'salt']
----------------------------------------
Top Fuzzy Match Name: Black Bean Salsa
Ingredients: ['black beans', 'tomatoes', 'roma tomatoes', 'green onions', 'garlic', 'cilantro', 'white pepper']
----------------------------------------
Top Fuzzy Match Name: Beviers BBQ Oysters With Habanero Salsa
Ingredients: ['salsa', 'white wine', 'lemon juice', 'Worcestershire sauce', 'roma tomatoes', 'sweet onion', 'bell pepper', 'habanero pepper', 'garlic cloves', 'lime juice', 'cilantro', 'kosher salt', 'pepper']
----------------------------------------
Top Fuzzy Match Name: Homemade Tomato Sauce
Ingredients: ['butter', 'onion', 'brown sugar', 'roma tomatoes', 'garlic cloves', 'tomato paste', 'extra virgin olive oil']
----------------------------------------
Top Fuzzy Match Name: Blue Ribbon Fresh